# TensorFlow / Keras

- Dense Deep Network using Keras API
- Tokenized Data instead of the vector of CountVectorizer with the size of vocab (better scalling)

1. [Keras](#Experiment-3-(Keras))

**Author:** BrenoAV

**Last Date Modified:** 2/2/2024

# Load Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv", sep="\t", encoding="utf-8")

In [3]:
df

,sentence,target,source
0,So there is no way for me to plug it in here i...,0,amazon
1,"Good case, Excellent value.",1,amazon
2,Great for the jawbone.,1,amazon
3,Tied to charger for conversations lasting more...,0,amazon
4,The mic is great.,1,amazon
...,...,...,...
2743,I think food should have flavor and texture an...,0,yelp
2744,Appetite instantly gone.,0,yelp
2745,Overall I was not impressed and would not go b...,0,yelp
2746,"The whole experience was underwhelming, and I ...",0,yelp


## Split dataset

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df["sentence"], df["target"], test_size=0.2, random_state=123)

In [6]:
X_train.shape, y_train.shape

((2198,), (2198,))

In [7]:
X_test.shape, y_test.shape

((550,), (550,))

# Preprocessing

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

2024-02-02 22:39:50.787848: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 22:39:50.817325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 22:39:50.817347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 22:39:50.817366: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 22:39:50.822464: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 22:39:50.822866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [9]:
NUM_WORDS = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_train)  # Important be only the train data!!!

In [10]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [11]:
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded = tokenizer.texts_to_sequences(X_test)

In [12]:
tokenizer.texts_to_sequences(["The mic is great."])

[[1, 1213, 5, 20]]

In [13]:
X_train_encoded = pad_sequences(X_train_encoded, maxlen=MAX_LEN, padding="post")
X_test_encoded = pad_sequences(X_test_encoded, maxlen=MAX_LEN, padding="post")

In [14]:
X_train_encoded.shape, X_test_encoded.shape

((2198, 100), (550, 100))

# MLFlow

In [15]:
import mlflow
from mlflow.data.pandas_dataset import PandasDataset

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [16]:
dataset: PandasDataset = mlflow.data.from_pandas(df, source="data.csv")

/home/brenoav/GitHub/NLP-Sentiment-Analysis/.venv/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/home/brenoav/GitHub/NLP-Sentiment-Analysis/.venv/lib/python3.10/site-packages/mlflow/data/digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


# Experiment 3 (Keras)

In [17]:
import tensorflow as tf
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [18]:
experiment_name = "sentiment_analysis_tf_dense"

experiment_tags = {
    "nlp.framework": "TensorFlow",
    "nlp.encoding": "Tokenizer",
    "nlp.model": "Dense Network",
    "nlp.task": "Sentiment Analysis"
}

mlflow.create_experiment(name=experiment_name,
                         artifact_location="mlartifacts",
                         tags=experiment_tags)

'367826324696094615'

In [19]:
class DenseHyperModel(kt.HyperModel):
    def build(self, hp):
        kernel_regularizer = hp.Choice("kernel_regularizer", ["None", "l2"])

        if kernel_regularizer.lower() == "none":
            kernel_regularizer = None

        model = Sequential(name="DenseNetwork")
        model.add(Embedding(input_dim=len(tokenizer.index_word) + 1, 
                            output_dim=hp.Choice("ouput_dim", [50, 75, 100]), 
                            input_length=MAX_LEN))
        model.add(Flatten())
        for i in range(hp.Int("num_layers", 1, 2)):
            model.add(Dense(units=hp.Choice("units", [16, 32, 64]), 
                            activation=hp.Choice("activation", ["relu", "tanh"]), 
                            kernel_regularizer=kernel_regularizer))
        model.add(Dense(units=1, activation="sigmoid"))

        model.compile(loss="binary_crossentropy", 
                      optimizer=Adam(learning_rate=hp.Choice("lr", [1e-2, 1e-3, 1e-4])),
                      metrics=["accuracy"])

        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            **kwargs,
        )

In [20]:
mlflow.set_experiment(experiment_name=experiment_name)  # It could be use the ID too

tuner = kt.RandomSearch(
    DenseHyperModel(),
    objective='val_accuracy',
    max_trials=30,
    seed=123,
    overwrite=True,
    directory="keras-tuner",
    project_name="sentiment_analysis"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x=X_train_encoded, y=y_train, 
             epochs=500,
             validation_data=(X_test_encoded, y_test),
             callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_encoded, 
                    y_train, 
                    validation_data=(X_test_encoded, y_test),
                    callbacks=[stop_early],
                    epochs=500,
)

y_pred = tf.math.round(model.predict(X_test_encoded))

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
precision = precision_score(y_true=y_test, y_pred=y_pred)
recall = recall_score(y_true=y_test, y_pred=y_pred)
f1 = f1_score(y_true=y_test, y_pred=y_pred)

metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1": f1
}

run_name = "_".join([f"{k}_{v}" for k, v in best_hps.values.items()])

with mlflow.start_run(run_name=run_name):
    # Log the hyperparameters
    mlflow.log_params(best_hps.values)

    # Log the metrics
    mlflow.log_metrics(metrics)
    ## Graph of accuracies and losses
    h = history.history
    for j, (train_loss, train_acc, test_loss, test_acc) in enumerate(zip(h["loss"], h["accuracy"], h["val_loss"], h["val_accuracy"])):
        mlflow.log_metric(key="train_loss", value=train_loss, step=j)
        mlflow.log_metric(key="train_acc", value=train_acc, step=j)
        mlflow.log_metric(key="test_loss", value=test_loss, step=j)
        mlflow.log_metric(key="test_acc", value=test_acc, step=j)

    # Log the dataset
    mlflow.log_input(dataset, context="training")

    # Log the model ! New versions it is mlflow.keras !
    mlflow.tensorflow.log_model(model=model,
                                artifact_path="mlartifacts",
                                input_example=X_train_encoded)

    mlflow.log_artifact("keras-tuner", artifact_path="mlartifacts")
    
    mlflow.log_artifact("tokenizer.pkl", artifact_path="mlartifacts")

Trial 30 Complete [00h 00m 08s]
val_accuracy: 0.8006060719490051

Best val_accuracy So Far: 0.8187878727912903
Total elapsed time: 00h 11m 09s
Epoch 1/500
69/69 [==============================] - 1s 4ms/step - loss: 0.9402 - accuracy: 0.4973 - val_loss: 0.8743 - val_accuracy: 0.5055
Epoch 2/500
69/69 [==============================] - 0s 3ms/step - loss: 0.8306 - accuracy: 0.5023 - val_loss: 0.7928 - val_accuracy: 0.4927
Epoch 3/500
69/69 [==============================] - 0s 3ms/step - loss: 0.7682 - accuracy: 0.5136 - val_loss: 0.7482 - val_accuracy: 0.4927
Epoch 4/500
69/69 [==============================] - 0s 3ms/step - loss: 0.7337 - accuracy: 0.5473 - val_loss: 0.7236 - val_accuracy: 0.5527
Epoch 5/500
69/69 [==============================] - 0s 3ms/step - loss: 0.7140 - accuracy: 0.5746 - val_loss: 0.7116 - val_accuracy: 0.5055
Epoch 6/500
69/69 [==============================] - 0s 3ms/step - loss: 0.7029 - accuracy: 0.6192 - val_loss: 0.7025 - val_accuracy: 0.5982
Epoch 7/500

/home/brenoav/GitHub/NLP-Sentiment-Analysis/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


INFO:tensorflow:Assets written to: /tmp/tmpinmaken4/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpinmaken4/model/data/model/assets
/home/brenoav/GitHub/NLP-Sentiment-Analysis/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


This Jupyter Notebook was created by BrenoAV. For any inquiries or feedback, please feel free to create an issue on [GitHub](https://github.com/BrenoAV/NLP-Sentiment-Analysis/issues) 📣.